# 패키지 로드

In [1]:
%pip install torch datasets
%pip install transformers accelerate peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 252.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 298.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 393.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 315.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 180.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 182.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 275.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pi

In [2]:
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

# 데이터 가져오기

In [3]:
import json

def load_train_dataset_from_json(path: str):
    """전체가 리스트 형태인 JSON 파일을 로드"""
    with open(path, "r", encoding="utf-8") as f:
        dataset = json.load(f)
    return dataset

# 사용 예시
train_dataset = load_train_dataset_from_json("qwen3_company_train_dataset_informal.json")


## 데이터 구조 확인

In [4]:
# 데이터셋 길이
len(train_dataset)

665

In [5]:
# 데이터셋 구조
train_dataset[:2]

[{'messages': [{'role': 'system',
    'content': '\n당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.\n다음 지침을 따르세요:\n1. 기존의 말투는 잊고 가볍고 친근한 반말로 답변하세요.\n2. 사실에 기반한 정보를 사용하세요.\n3. 답변이 불확실한 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.\n4. 답변이 너무 길지 않게 하세요.\n'},
   {'role': 'user',
    'content': '서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?\n검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->\n# 서비스 아키텍처 문서\n분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29\n\n## 1. 개요\n코드노바의 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI 페르소나 챗봇 앱인 크랙(Crack), 그리고 대화형 광고 제작·보상 플랫폼인 Wrtn Ads를 지원하도록 설계되었습니다. 이 문서는 백엔드 시스템의 구성 요소와 상호작용을 설명합니다.\n\n## 2. 아키텍처 구성 요소\n\n### 2.1. API 서버\n- **역할**: 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환합니다.\n- **기술 스택**: Node.js, Express.js\n- **검증 포인트**:\n  - API 엔드포인트가 올바르게 작동하는지 확인\n  - 요청 처리 속도 및 오류율 모니터링 [[ref1]]\n## 7. 문서화\n- **API 문서화**: 모든 API 엔드포인트는 Swagger 또는 Postman과 같은 도구를 사용하여 문서화되어야 하며, 최신 상태를 유지해야 합니다.\n- **사용자 가이드**: 외부 개발자와 내부 팀을 위한 사용자 가이드를 작성하여 API 사용 방법과 예제를 제공해야 합니다.\n\n## 8.

## 리스트 형태에서 Dataset 객체로 변경

In [6]:
print(type(train_dataset))

train_dataset = Dataset.from_list(train_dataset)

print(type(train_dataset))

<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>


# 모델 로드 및 템플릿 적용

In [7]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen3-8B"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [8]:
# 템플릿 적용 확인
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system

당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 가볍고 친근한 반말로 답변하세요.
2. 사실에 기반한 정보를 사용하세요.
3. 답변이 불확실한 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
4. 답변이 너무 길지 않게 하세요.
<|im_end|>
<|im_start|>user
서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?
검색 결과:
-----
<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->
# 서비스 아키텍처 문서
분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

## 1. 개요
코드노바의 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI 페르소나 챗봇 앱인 크랙(Crack), 그리고 대화형 광고 제작·보상 플랫폼인 Wrtn Ads를 지원하도록 설계되었습니다. 이 문서는 백엔드 시스템의 구성 요소와 상호작용을 설명합니다.

## 2. 아키텍처 구성 요소

### 2.1. API 서버
- **역할**: 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환합니다.
- **기술 스택**: Node.js, Express.js
- **검증 포인트**:
  - API 엔드포인트가 올바르게 작동하는지 확인
  - 요청 처리 속도 및 오류율 모니터링 [[ref1]]
## 7. 문서화
- **API 문서화**: 모든 API 엔드포인트는 Swagger 또는 Postman과 같은 도구를 사용하여 문서화되어야 하며, 최신 상태를 유지해야 합니다.
- **사용자 가이드**: 외부 개발자와 내부 팀을 위한 사용자 가이드를 작성하여 API 사용 방법과 예제를 제공해야 합니다.

## 8. 보안
- **데이터 보호**: API를 통해 전송되는 모든 데이터는 암호화되어야 하며, 민감한 정보는 절대 노출되지 않도록 해야

# LoRA와 SFTConfig 설정

In [9]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [10]:
args = SFTConfig(
    output_dir="qwen3-8b-informal",          # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=[]
)

# 학습 중 전처리 함수: collate_fn

In [11]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)

        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()

        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=8192,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        # 레이블 초기화
        labels = [-100] * len(input_ids)

        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"

        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)

        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):

                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):

                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)

                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1

                    i = current_pos

            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])

    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])

        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)

    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)

    return new_batch

# 학습

In [12]:
# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 1665])
어텐션 마스크 형태: torch.Size([1, 1665])
레이블 형태: torch.Size([1, 1665])


In [13]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

In [14]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,0.973200
20,0.693300
30,0.473200
40,0.372100
50,0.350800
60,0.272100
70,0.344300
80,0.290600
90,0.269800
100,0.283900


# 파인 튜닝 모델 테스트

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [16]:
class QwenChatbot:
    def __init__(self,
                 model_name="Qwen/Qwen3-8B",
                 lora_path=None,
                 system_message="You are a helpful AI assistant."):

        # 토크나이저 및 기본 모델 로드
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        base_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto"
        )

        # LoRA 적용
        if lora_path:
            print(f"LoRA 어댑터 로드: {lora_path}")
            self.model = PeftModel.from_pretrained(base_model, lora_path)
        else:
            self.model = base_model

        self.history = []

        # 시스템 메시지 초기화
        if system_message:
            self.history.append({"role": "system", "content": system_message})

    def generate_response(self, user_input, max_new_tokens=1024):
        # 대화 기록 + 사용자 입력
        messages = self.history + [{"role": "user", "content": user_input}]

        # Chat 템플릿 적용
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # 모델 입력 변환
        inputs = self.tokenizer(text, return_tensors="pt").to(self.model.device)

        # 응답 생성
        response_ids = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
        )[0][len(inputs.input_ids[0]):].tolist()

        response = self.tokenizer.decode(response_ids, skip_special_tokens=True)

        # 히스토리 업데이트
        self.history.append({"role": "user", "content": user_input})
        self.history.append({"role": "assistant", "content": response})

        return response

In [17]:
tone_instruction = f"기존의 말투는 잊고 가볍고 친근한 반말로 답변하세요."

# 새 시스템 메시지
system_message = f"""
당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. {tone_instruction}
2. 사실에 기반한 정보를 사용하세요.
3. 답변이 불확실한 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
4. 답변이 너무 길지 않게 하세요.
"""

In [18]:
# Example Usage
if __name__ == "__main__":
    chatbot = QwenChatbot(
        model_name="Qwen/Qwen3-8B",
        lora_path="qwen3-8b-informal/checkpoint-501",
        system_message=system_message
    )

    # First input
    user_input_1 = "서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?"
    print(f"User: {user_input_1}")
    response_1 = chatbot.generate_response(user_input_1)
    print(f"Bot: {response_1}")
    print("----------------------")

    # Second input
    user_input_2 = "API 서버의 검증 포인트에 대한 구체적인 테스트 방법은 무엇인가요?"
    print(f"User: {user_input_2}")
    response_2 = chatbot.generate_response(user_input_2)
    print(f"Bot: {response_2}")
    print("----------------------")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


LoRA 어댑터 로드: qwen3-8b-informal/checkpoint-501
User: 서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?
Bot: <think>

</think>

서비스 아키텍처 문서에서 API 서버의 검증 포인트는 다음과 같아:

1. 인증 검증: 사용자의 신분을 확인하는 과정이야.
2. 권한 검증: 사용자가 특정 리소스에 접근할 수 있는 권한을 확인하는 거야.
3. 입력 데이터 검증: 클라이언트가 보낸 데이터의 유효성을 검사해서 부적절한 데이터를 거부해.
4. 보안 헤더 검증: 요청에 포함된 보안 헤더를 검사해서 보안 위협을 방지해.

이런 검증 포인트는 API 서버의 보안과 신뢰성을 높이는 데 중요한 역할을 해.
----------------------
User: API 서버의 검증 포인트에 대한 구체적인 테스트 방법은 무엇인가요?
Bot: <think>

</think>

API 서버의 검증 포인트에 대한 구체적인 테스트 방법은 다음과 같아:

1. **인증 검증 테스트**: 다양한 인증 방법(예: OAuth, JWT)을 사용해서 인증 토큰이 올바르게 생성되고 검증되는지 확인해.
2. **권한 검증 테스트**: 사용자가 특정 리소스에 접근할 수 있는 권한을 테스트하기 위해, 다양한 권한 수준을 설정하고 해당 권한을 가진 사용자가 요청을 성공적으로 처리하는지 확인해.
3. **입력 데이터 검증 테스트**: 다양한 입력 데이터를 생성해서 API가 이를 올바르게 검증하고 부적절한 데이터를 거부하는지 테스트해.
4. **보안 헤더 검증 테스트**: 요청에 포함된 보안 헤더(예: Content-Security-Policy, X-Content-Type-Options)를 검사해서 보안 위협을 방지하는지 확인해.

이런 테스트 방법들은 API 서버의 검증 포인트가 올바르게 작동하는지 확인하는 데 도움이 돼.
----------------------


# 모델 병합

In [19]:
from peft import AutoPeftModelForCausalLM

peft_model_id = "qwen3-8b-informal/checkpoint-501"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", dtype=torch.float16)

# LoRA → base weight에 병합
merged_model = fine_tuned_model.merge_and_unload()

# 허브에 업로드
repo_id = "SKN14-Final-1Team/qwen3-8b-informal-merged"
merged_model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...sw/model-00004-of-00004.safetensors:   1%|1         | 16.7MB / 1.58GB            

  ...sw/model-00003-of-00004.safetensors:   2%|1         | 83.8MB / 4.98GB            

...

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpflpob55h/tokenizer.json       : 100%|##########| 11.4MB / 11.4MB            

  /tmp/tmpflpob55h/tokenizer.json       : 100%|##########| 11.4MB / 11.4MB            

CommitInfo(commit_url='https://huggingface.co/SKN14-Final-1Team/qwen3-8b-informal-merged/commit/5cea132422226f3e2e633e05f48bb45cc7d91788', commit_message='Upload tokenizer', commit_description='', oid='5cea132422226f3e2e633e05f48bb45cc7d91788', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SKN14-Final-1Team/qwen3-8b-informal-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='SKN14-Final-1Team/qwen3-8b-informal-merged'), pr_revision=None, pr_num=None)